<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
💰 תהליך הסבת שכבת גושי שומה לבנק"ל מודרני 💰 

In [ ]:
print('1. Tax Blocks Borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  1.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
מספרי גושי שומא מתוך טבלת גושי שומא

In [ ]:
table_cols = ['GUSH_NUM']

Tax_Table = pd.read_csv(CNFG.Oracle +'gush_shuma.csv', usecols = table_cols, encoding = 'cp1255')

<h1 style="text-align:right;font-size:100%">
שכבת גושי שומא מבוטלים (לפי חיתוך מול גושי קדסטר)

In [ ]:
layer_cols = ['GUSH_NUM', 'geometry']

Tax_Cancel = gpd.read_file(CNFG.CADSDE + 'SUB_GUSH_ALL_SHUMA.gdb', layer = 'SUB_GUSH_ALL_SHUMA_cancel', include_fields = layer_cols)

<h1 style="text-align:right;font-size:100%">
שכבת גושי שומא ברצף (לפי חיתוך מול גושי קדסטר)

In [ ]:
Tax_remains = gpd.read_file(CNFG.CADSDE + 'SUB_GUSH_ALL_SHUMA.gdb', layer = 'SUB_GUSH_ALL_SHUMA_remains', include_fields = layer_cols)

In [ ]:
del[table_cols, layer_cols]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️
<h1 style="text-align:right; font-size:125%">
<h1 style="text-align:right; font-size:100%">
הגדרת שדה שמתאר האם הגוש מבוטל או לא
<h1 style="text-align:right; font-size:100%">
איחוד גושי שומא מבוטלים עם רציפים
<h1 style="text-align:right; font-size:100%">
סינון טבלת גושי שומא לגושי שומא שלא נמצאים בשכבה המאוחדת
<h1 style="text-align:right; font-size:100%">
איחוד של הרשימה עם האיחוד הקודם

In [ ]:
print('  1.b Cleaning data')

In [ ]:
Tax_remains['IsCancel'] = False
Tax_Cancel['IsCancel'] = True

In [ ]:
Tax_Layer = pd.concat([Tax_remains, Tax_Cancel], ignore_index = True)

In [ ]:
Tax_Table = Tax_Table[~Tax_Table['GUSH_NUM'].isin(Tax_Layer['GUSH_NUM'])]
Tax_Layer = pd.concat([Tax_Layer, Tax_Table], ignore_index = True)

del Tax_Table

In [ ]:
print('  1.c Data conversion')

<h1 style="text-align:right; font-size:100%">
שינוי שמות שדות

In [ ]:
Tax_Layer.rename(columns={'GUSH_NUM':'BlockNumber', 'GUSH_SUFFIX':'SubBlockNumber'}, inplace = True)

<h1 style="text-align:right; font-size:100%">
הגדרת שדה שם מלא לגוש (כל תתי הגוש הם 0)

In [ ]:
Tax_Layer['Name'] = Tax_Layer['BlockNumber'].astype(str) + '/0'

<h1 style="text-align:right; font-size:100%">
הגדרת שדות של סוגי ומזהי תהליכים יוצרים ומבטלים

In [ ]:
Tax_Layer['CreatedByRecord'] = CNFG.FabricatedCreateProcessGUID


retired_cond = (Tax_Layer['IsCancel'] == True) | (Tax_Layer['geometry'].isna())
Tax_Layer['RetiredByRecord'] = np.where(retired_cond, CNFG.FabricatedCancelProcessGUID, None)
Tax_Layer.drop(columns = 'IsCancel', inplace = True)

<h1 style="text-align:right; font-size:100%">
הגדרת שדות נוספים להשלמת הטבלה

In [ ]:
Tax_Layer['SubBlockNumber'] = 0

In [ ]:
Tax_Layer['BlockStatus'] = 21 # סטאטוס לא מוסדר

In [ ]:
Tax_Layer['IsTax'] = 1 # כן

In [ ]:
Tax_Layer['LandType'] = 2 #לא מוסדר

In [ ]:
Tax_Layer['IsJordanian'] = 0 # לא ירדני

In [ ]:
Tax_Layer['StatedAreaUnit'] = 109404 #Square Meter

In [ ]:
Tax_Layer[['SetteledDate', 'UpdateDate', 'LastRegisterdParcel', 'LastParcel', 'LastSetteledParcel', 'LastCourtParcel']] = None

In [ ]:
columns_order = ['geometry', 'BlockNumber', 'SubBlockNumber',
                 'Name', 'BlockStatus', 'StatedAreaUnit', 'IsJordanian', 'LandType',
                 'SetteledDate', 'UpdateDate',
                 'LastRegisterdParcel', 'LastParcel', 'LastSetteledParcel', 'LastCourtParcel',
                 'CreatedByRecord', 'RetiredByRecord', 'IsTax']



Tax_Layer = Tax_Layer[columns_order]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
הוספת גושי השומא לתוצאות כל הגושים
<h1 style="text-align:right; font-size:100%;"> 
הפעלת יישום להכנסת כל הגושים לטבלת גושים בבנק"ל

In [ ]:
print('  1.e Exporting results')

In [ ]:
Tax_Layer.to_file(CNFG.Results, layer = 'Blocks', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'a')

In [ ]:
print('  1.f  Appending results to Blocks')

In [ ]:
subprocess.call([CNFG.BatchPath + 'Blocks-Save as Feature Class.bat']);